In [11]:
%reload_ext autoreload
%autoreload 2

import colorcet as cc
import holoviews as hv
import hvplot.polars  # noqa
import numpy as np
import polars as pl
from bokeh import io
from scipy import stats
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from sklearn import linear_model
from webdriver_manager.firefox import GeckoDriverManager

from lib import ParameterDataset, RuntimeDataset

pl.Config.set_tbl_rows(50)

driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))

## Runtime basline experiment

**Objective**: Determine if the runtime across distribution types is the same.

We cannot use ANOVA because the samples are not normally distributed.

https://www.pythonfordatascience.org/parametric-assumptions-python

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.f_oneway.html#scipy.stats.f_oneway

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html#scipy.stats.shapiro

In [7]:
df = lib.load_dataset('runtime-baseline')
network_types = lib.get_network_types(df)

NameError: name 'lib' is not defined

In [ ]:
for network_type in network_types:
    data = lib.compute_runtime_results(df, network_type).flatten()
    print(network_type, '\t', stats.shapiro(data))

We can use the nonparametric test instead to compare the medians across samples.

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kruskal.html#scipy.stats.kruskal

In [ ]:
for network_type in network_types:
    samples = lib.compute_runtime_results(df, network_type)
    print(network_type, '\t', stats.kruskal(*samples))

## Send coefficient experiment

In [47]:
df = ParameterDataset.load(name='send-coefficient', parameter='send_coefficient')

In [48]:
accuracy = df.accuracy_results(percentiles=[0, 0.01, 0.1, *range(1, 7)])
accuracy

send_coefficient,frequency,accuracy_p0,accuracy_p0.01,accuracy_p0.1,accuracy_p1,accuracy_p2,accuracy_p3,accuracy_p4,accuracy_p5,accuracy_p6,normalized_frequency
f64,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1.0,11258,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1.1,2899,0.944608,0.964611,0.979388,0.993281,0.997386,0.999814,1.0,1.0,1.0,0.980985
1.2,2622,0.916469,0.951758,0.971798,0.989989,0.995163,0.998132,1.0,1.0,1.0,0.976088
1.3,2535,0.88289,0.937096,0.961036,0.98634,0.992565,0.996313,0.99898,1.0,1.0,0.971659
1.4,2465,0.882113,0.922487,0.95256,0.981569,0.9899,0.994546,0.997636,1.0,1.0,0.967378
1.5,2622,0.860312,0.910898,0.946823,0.979661,0.988392,0.993467,0.996968,0.999669,1.0,0.963214
1.6,2392,0.855067,0.90064,0.939854,0.976501,0.986631,0.992109,0.995943,0.998952,1.0,0.958785
1.7,2329,0.847622,0.890528,0.934432,0.973824,0.985078,0.991087,0.995385,0.998617,1.0,0.954745
1.8,2856,0.840172,0.88372,0.930491,0.97133,0.983497,0.989901,0.994547,0.998071,1.0,0.950811


In [49]:
accuracy_nt = df.accuracy_results(percentiles=[0, 0.01, 0.1, *range(1, 12)], by_network_type=True)
accuracy_nt

network_type,send_coefficient,frequency,accuracy_p0,accuracy_p0.01,accuracy_p0.1,accuracy_p1,accuracy_p2,accuracy_p3,accuracy_p4,accuracy_p5,accuracy_p6,accuracy_p7,accuracy_p8,accuracy_p9,accuracy_p10,accuracy_p11,normalized_frequency
str,f64,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""BarabasiAlbert""",1.0,7647,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"""BarabasiAlbert""",1.1,1169,0.950307,0.959333,0.973929,0.988234,0.992071,0.994533,0.99635,0.99766,0.998739,0.9997,1.0,1.0,1.0,1.0,0.947833
"""BarabasiAlbert""",1.2,880,0.916469,0.948319,0.96306,0.983075,0.988704,0.991763,0.994108,0.995937,0.997346,0.99857,0.999682,1.0,1.0,1.0,0.939859
"""BarabasiAlbert""",1.3,941,0.88289,0.927868,0.949986,0.9779,0.985225,0.989179,0.991719,0.994012,0.995881,0.997478,0.99878,1.0,1.0,1.0,0.933855
"""BarabasiAlbert""",1.4,868,0.883863,0.907635,0.940443,0.969804,0.979543,0.985391,0.989104,0.992042,0.994533,0.996278,0.997793,0.999152,1.0,1.0,0.927436
"""BarabasiAlbert""",1.5,868,0.8616,0.892708,0.931439,0.96626,0.978125,0.983806,0.987807,0.990849,0.993401,0.995587,0.997273,0.998848,1.0,1.0,0.921515
"""BarabasiAlbert""",1.6,787,0.855067,0.873413,0.921867,0.962369,0.974672,0.981378,0.985828,0.989146,0.991974,0.994493,0.996459,0.998288,0.999934,1.0,0.915593
"""BarabasiAlbert""",1.7,812,0.847622,0.872279,0.911754,0.95875,0.971658,0.97908,0.984338,0.988042,0.991238,0.994017,0.996234,0.998134,0.99978,1.0,0.910225
"""BarabasiAlbert""",1.8,780,0.840172,0.869234,0.907749,0.954759,0.96796,0.977073,0.982503,0.986757,0.989966,0.992764,0.995358,0.997441,0.999354,1.0,0.904685


In [50]:
melted = (
    accuracy
    # .filter(pl.col('network_type').eq('RandomRegular'))
    .drop('network_type', 'frequency', 'normalized_frequency')
    .melt(id_vars=['send_coefficient'], variable_name='percentile', value_name='accuracy')
    .with_columns(pl.col('percentile').str.strip_prefix("accuracy_p").cast(pl.Float64))
)

In [51]:
box = melted.hvplot.box(by='percentile', y='accuracy', box_fill_alpha=0)

scatter = melted.hvplot.scatter(
    x='percentile', 
    y='accuracy', 
    c='send_coefficient', 
    cmap=cc.b_linear_kry_5_98_c75,
    clabel='Send coefficient', 
    size=10
).opts(jitter=0.0)

plot = (box * scatter).opts(
    show_legend=False, 
    xlabel='Percentile', 
    ylabel='Accuracy',
    backend_opts={"plot.toolbar.autohide": True}
)

io.export_png(hv.render(plot), filename='accuracy-box.png', webdriver=driver)

plot

:Overlay
   .BoxWhisker.I :BoxWhisker   [percentile]   (accuracy)
   .Scatter.I    :Scatter   [percentile]   (accuracy,send_coefficient)

In [79]:
min_parameter = 1

efficiency = df.efficiency_results(min_parameter=min_parameter)
efficiency_nt = df.efficiency_results(min_parameter=min_parameter, by_network_type=True)

In [82]:
efficiency.hvplot.box(by='send_coefficient')

:BoxWhisker   [send_coefficient]   (n_receives)

In [61]:
efficiency_nt

network_type,send_coefficient,n_receives_p0,n_receives_p10,n_receives_p25,n_receives_p50,n_receives_p75,n_receives_p90,n_receives_p100
str,f64,f64,f64,f64,f64,f64,f64,f64
"""BarabasiAlbert""",0.8,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"""BarabasiAlbert""",0.9,0.399583,0.420862,0.441937,0.470031,0.510774,0.553901,0.586463
"""BarabasiAlbert""",1.0,0.299155,0.321306,0.34055,0.362645,0.391622,0.434981,0.459019
"""BarabasiAlbert""",1.1,0.256961,0.280473,0.295424,0.313707,0.339137,0.379033,0.404236
"""BarabasiAlbert""",1.2,0.246861,0.264231,0.279251,0.298689,0.323664,0.358407,0.368705
"""BarabasiAlbert""",1.3,0.237233,0.255513,0.270611,0.286189,0.310007,0.344679,0.359866
"""BarabasiAlbert""",1.4,0.230157,0.249593,0.260897,0.280453,0.302899,0.336838,0.347779
"""BarabasiAlbert""",1.5,0.225875,0.245242,0.25978,0.273,0.295296,0.331784,0.346753
"""BarabasiAlbert""",1.6,0.223035,0.242742,0.256889,0.271294,0.295038,0.323924,0.343016


In [51]:
def percentiles(df, metric, percentiles=(0, 10, 25, 50, 75, 90, 95, 99, 100),
                group_by=('network_type', 'send_coefficient')):
    return (
        df
        .group_by(*group_by)
        .agg(**{f'$$P_{ {p} }$$': pl.col(metric).quantile(p / 100) for p in percentiles})
        .sort(group_by)
        .collect()
    )

In [57]:
percentiles(df, 'msg_reachability', group_by=('network_type', 'tolerance'))

network_type,$$P_{0}$$,$$P_{10}$$,$$P_{25}$$,$$P_{50}$$,$$P_{75}$$,$$P_{90}$$,$$P_{95}$$,$$P_{99}$$,$$P_{100}$$
str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""BarabasiAlbert""",0.0,0.0,0.0,0.0,1.0,2.0,3.0,4.0,7.0
"""GnmRandom""",0.0,0.0,0.0,0.0,2.0,2.0,3.0,4.0,8.0
"""RandomRegular""",0.0,0.0,0.0,0.0,2.0,2.0,3.0,4.0,8.0
"""WattsStrogatz""",0.0,0.0,0.0,1.0,2.0,3.0,3.0,4.0,9.0


In [53]:
percentiles(df, 'msg_reachability', group_by=['network_type'])

network_type,$$P_{0}$$,$$P_{10}$$,$$P_{25}$$,$$P_{50}$$,$$P_{75}$$,$$P_{90}$$,$$P_{95}$$,$$P_{99}$$,$$P_{100}$$
str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""BarabasiAlbert""",0.0,0.0,0.0,0.0,1.0,2.0,3.0,4.0,7.0
"""GnmRandom""",0.0,0.0,0.0,0.0,2.0,2.0,3.0,4.0,8.0
"""RandomRegular""",0.0,0.0,0.0,0.0,2.0,2.0,3.0,4.0,8.0
"""WattsStrogatz""",0.0,0.0,0.0,1.0,2.0,3.0,3.0,4.0,9.0


In [55]:
percentiles(df, 'msg_reachability', group_by=['tolerance'])

tolerance,$$P_{0}$$,$$P_{10}$$,$$P_{25}$$,$$P_{50}$$,$$P_{75}$$,$$P_{90}$$,$$P_{95}$$,$$P_{99}$$,$$P_{100}$$
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.001,0.0,0.0,0.0,0.0,2.0,2.0,3.0,4.0,9.0
0.002,0.0,0.0,0.0,0.0,2.0,2.0,3.0,4.0,9.0
0.003,0.0,0.0,0.0,0.0,2.0,2.0,3.0,4.0,9.0
0.004,0.0,0.0,0.0,0.0,2.0,2.0,3.0,4.0,9.0
0.005,0.0,0.0,0.0,0.0,2.0,2.0,3.0,4.0,9.0
0.006,0.0,0.0,0.0,0.0,2.0,2.0,3.0,4.0,9.0
0.007,0.0,0.0,0.0,0.0,2.0,2.0,3.0,4.0,8.0
0.008,0.0,0.0,0.0,0.0,2.0,2.0,3.0,4.0,9.0
0.009,0.0,0.0,0.0,0.0,2.0,2.0,3.0,4.0,8.0


In [58]:
percentiles(df, 'n_influenced')

network_type,send_coefficient,$$P_{0}$$,$$P_{10}$$,$$P_{25}$$,$$P_{50}$$,$$P_{75}$$,$$P_{90}$$,$$P_{95}$$,$$P_{99}$$,$$P_{100}$$
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""BarabasiAlbert""",1.0,0.0,0.0,0.0,0.0,9.0,47.0,107.0,446.0,4413.0
"""GnmRandom""",1.0,0.0,0.0,0.0,0.0,15.0,70.0,145.0,394.0,2119.0
"""RandomRegular""",1.0,0.0,0.0,0.0,0.0,16.0,72.0,145.0,383.0,1809.0
"""WattsStrogatz""",1.0,0.0,0.0,0.0,3.0,19.0,57.0,102.0,256.0,1675.0


In [59]:
percentiles(df, 'n_influenced', group_by=['network_type'])

network_type,$$P_{0}$$,$$P_{10}$$,$$P_{25}$$,$$P_{50}$$,$$P_{75}$$,$$P_{90}$$,$$P_{95}$$,$$P_{99}$$,$$P_{100}$$
str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""BarabasiAlbert""",0.0,0.0,0.0,0.0,9.0,47.0,107.0,446.0,4413.0
"""GnmRandom""",0.0,0.0,0.0,0.0,15.0,70.0,145.0,394.0,2119.0
"""RandomRegular""",0.0,0.0,0.0,0.0,16.0,72.0,145.0,383.0,1809.0
"""WattsStrogatz""",0.0,0.0,0.0,3.0,19.0,57.0,102.0,256.0,1675.0


In [60]:
percentiles(df, 'n_influenced', group_by=['tolerance'])

tolerance,$$P_{0}$$,$$P_{10}$$,$$P_{25}$$,$$P_{50}$$,$$P_{75}$$,$$P_{90}$$,$$P_{95}$$,$$P_{99}$$,$$P_{100}$$
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.001,0.0,0.0,0.0,0.0,15.0,62.0,127.0,372.0,4067.0
0.002,0.0,0.0,0.0,0.0,15.0,61.0,126.0,371.0,4413.0
0.003,0.0,0.0,0.0,0.0,15.0,61.0,126.0,368.0,4317.0
0.004,0.0,0.0,0.0,0.0,15.0,61.0,125.0,365.0,4321.0
0.005,0.0,0.0,0.0,0.0,15.0,61.0,125.0,364.0,4281.0
0.006,0.0,0.0,0.0,0.0,15.0,61.0,125.0,361.0,4154.0
0.007,0.0,0.0,0.0,0.0,15.0,60.0,124.0,358.0,3775.0
0.008,0.0,0.0,0.0,0.0,15.0,60.0,124.0,359.0,4231.0
0.009,0.0,0.0,0.0,0.0,15.0,60.0,123.0,355.0,3766.0


In [ ]:
percentiles(df, 'n_influences')

In [ ]:
percentiles(df, 'n_influences', group_by=['network_type'])

In [ ]:
percentiles(df, 'n_influences', group_by=['send_coefficient'])

In [83]:
(
    df
    .filter(pl.col.network_type == 'BarabasiAlbert')
    .hvplot.hist(y='msg_reachability', by='send_coefficient', subplots=True, width=250, height=250, xlim=(-0.5, 7.5),
                 shared_axes=False)
    .cols(3)
)

:NdLayout   [send_coefficient]
   :Histogram   [msg_reachability]   (Count)

In [27]:
(
    df
    .filter(pl.col.network_type == 'RandomRegular')
    .hvplot.hist(y='msg_reachability', by='send_coefficient', subplots=True, width=250, height=250, xlim=(-0.5, 7.5),
                 shared_axes=False)
    .cols(3)
)

:NdLayout   [send_coefficient]
   :Histogram   [msg_reachability]   (msg_reachability_count)

In [ ]:
(
    df
    .filter(pl.col.network_type == 'GnmRandom')
    .hvplot.hist(y='msg_reachability', by='send_coefficient', subplots=True, width=250, height=250, xlim=(-0.5, 7.5),
                 shared_axes=False)
    .cols(3)
)

In [ ]:
(
    df
    .filter(pl.col.network_type == 'WattsStrogatz')
    .hvplot.hist(y='msg_reachability', by='send_coefficient', subplots=True, width=250, height=250, xlim=(-0.5, 7.5),
                 shared_axes=False)
    .cols(3)
)

In [29]:
(
    df
    .hvplot.hist(y='msg_reachability', by='send_coefficient', subplots=True, width=250, height=250, xlim=(-0.5, 7.5),
                 shared_axes=False)
    .cols(3)
)

:NdLayout   [send_coefficient]
   :Histogram   [msg_reachability]   (msg_reachability_count)

In [28]:
(
    df
    .hvplot.kde(
        y='n_influenced',
        by='send_coefficient',
        subplots=True,
        shared_axes=True,
        xlim=(0, 100),
        xlabel='Influence set cardinality',
        ylabel='Density',
        clabel='Network type',
        fill_alpha=0.5,
        height=250,
        width=250,
    ).cols(3)
)

:NdLayout   [send_coefficient]
   :Distribution   [n_influenced]   (Density)

In [30]:
(
    df
    .filter(pl.col.network_type == 'WattsStrogatz')
    .hvplot.kde(
        y='n_influenced',
        by='send_coefficient',
        # subplots=True,
        # shared_axes=False,
        xlim=(0, 100),
        xlabel='Influence set cardinality',
        ylabel='Density',
        clabel='Network type',
        fill_alpha=0.5,
    )
)

:NdOverlay   [send_coefficient]
   :Distribution   [n_influenced]   (Density)

In [31]:
(
    df
    .hvplot.kde(
        y='n_influenced',
        by='network_type',
        xlim=(0, 100),
        xlabel='Influence set cardinality',
        ylabel='Density',
        clabel='Network type',
        color=[cc.CET_CBTL4[int(i)] for i in np.linspace(0, 255, 4)]
    )
)

:NdOverlay   [network_type]
   :Distribution   [n_influenced]   (Density)

In [33]:
(
    df
    .hvplot.kde(
        y='n_influences',
        by='network_type',
        xlim=(0, 100),
        xlabel='Source set cardinality',
        ylabel='Density',
        clabel='Network type',
        color=[cc.CET_CBTL4[int(i)] for i in np.linspace(0, 255, 4)]
    )
)

:NdOverlay   [network_type]
   :Distribution   [n_influences]   (Density)

In [35]:
df = RuntimeDataset.load('runtime')

In [37]:
runtimes = (
    df
    .select('msg_runtime', 'total_runtime')
    .with_columns(runtime_percent=pl.col.msg_runtime / pl.col.total_runtime)
)

In [38]:
df

statistic,msg_runtime,total_runtime,runtime_percent
str,f64,f64,f64
"""count""",2400.0,2400.0,2400.0
"""null_count""",0.0,0.0,0.0
"""mean""",90336.824583,126202.154167,-1517.496175
"""std""",87176.761339,73282.369497,14918.549794
"""min""",7633.0,-1.0,-318866.0
"""25%""",17830.0,75435.0,0.222664
"""50%""",61712.0,101481.0,0.567458
"""75%""",139128.0,154339.0,0.905731
"""max""",596961.0,613745.0,0.98626


In [42]:
df.filter(pl.col.n_nodes == 10_000).hvplot.scatter(
    x='n_edges',
    y='msg_runtime',
    groupby='network_type',
    color='n_nodes',
    # subplots=True, 
    # width=500, 
    # height=250, 
    # size=5
)

:DynamicMap   [network_type]
   :Scatter   [n_edges]   (msg_runtime,n_nodes)

In [ ]:
data = (
    df
    .select('n_nodes', 'n_edges', 'msg_runtime')
    .collect()
    .to_numpy()
)

x = data[:, :-1]
y = data[:, -1]

In [ ]:
reg = linear_model.Ridge(alpha=20)

In [ ]:
reg.fit(x, y)

In [ ]:
reg.coef_

In [ ]:
nodes = np.arange(1e2, 1e6, 100)
edges = np.arange(1e2, 1e6, 100)

In [ ]:
df = pl.DataFrame({
    'nodes': np.arange(1e2, 1e6, 100),
    'edges': np.arange(1e2, 1e6, 100)
})

In [ ]:
reg.score(x, y)

In [ ]:
df = df.with_columns(runtime=reg.intercept_ + pl.col.nodes * reg.coef_[0] + pl.col.edges * reg.coef_[1])

In [ ]:
df.hvplot.scatter(x='edges', y='runtime')

In [ ]:
import seaborn as sns

In [ ]:
sns.regplot(x=x[:, -1], y=y);

In [ ]:
x